In [117]:
# -------------------------------------------- Base FBA package --------------------------------------------
# -*- coding: utf-8 -*-

from __future__ import absolute_import

#import logging
import cobra

bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

#Adding a few exception classes to handle different types of errors
class FeasibilityError(Exception):
    """Error in FBA formulation
    """
    if Exception == '':
        pass
    
    if Exception == '':
        pass
    

#Base class for FBA packages
class BaseFBAPkg:
    def __init__(self, model, name, variable_types = {'concentration': 'float', 'lnconc': 'float'}, constraint_types = {'concentration': 'float'}, parent=None):
        '''Intantiate the model
        'model' (COBRA object): The COBRApy model object
        'name' (Python object, string): The name of the model
        'variable_types' (Python object, dictionary): The types and variables examples for the model variables
        'constraint_types' (Python object, dictionary): The names and values of the model constraints
        'parent' (Python object, boolean): The categorical description of the model 
        '''
        self.model = model
        self.name = name
        self.childpkgs = dict()
        self.parentpkg = parent
        self.constraints = dict()
        self.variables = dict()
        self.parameters = dict()
        self.variable_types = variable_types
        self.constraint_types = constraint_types
        for type in variable_types:
            self.variables[type] = dict()
        for type in constraint_types:
            self.constraints[type] = dict()
    
    
    def validate_parameters(self, params, required, defaults):
        '''Validate the parameters of the model
        'params' (Python object, dictionary): The parameters and values of the model
        'required' (Python object, list): The required parameters for the model 
        'defaults' (Python object, dictionary): The default parameters and values for the model 
        '''
        missing_parameters = set(required) - set(params)
        missing_string = ', '.join(list(missing_parameters))
        if len(missing_parameters) > 1:
            raise ValueError('The required parameters < {} > are missing!'.format(missing_string))
        elif len(missing_parameters) == 1:
            raise ValueError('The required parameter < {} > is missing!'.format(missing_string))
        elif len(missing_parameters) == 0:
            print('All required parameters are present.')
        
        self.parameters = params
        for key in defaults:
            if key not in params:
                self.parameters[key] = defaults[key]
        
        
    def build_variable(self, type, lower_bound, upper_bound, vartype, object):
        '''Create variables of the specified type that will be added to the COBRA model
        'type' (Python object, string): The variable type within which variables will be created
        'lower_bound' (Python object, float): The lower bound value for the added variable
        'upper_bound' (Python object, float): The upper bound value for the added variable
        'vartype' (Python object, string): The variable type as either 'continuous', 'integer', or 'binary' 
        'object' (Python object, string): The variable name when the name is defined
        '''
        # assign a variable name based upon the passed arguments
        if self.variable_types[type] == "none":
            count = len(self.variables[type])
            name = str(count + 1)
        elif self.variable_types[type] == "string":
            name = object
        elif vartype not in self.variable_types:
            self.variable_types[type] = vartype  
            name = object.id
            
        # add an optlang variable, when the variable is undefined
        #raise ValueError('The object name {} is not recognized by your model'.format(missing_string))
        if name in self.variables[type]:
            print('ERROR: The {} variable value of {} is being overwritten.'.format(name, self.variables[type][name]))
            self.variables[name] = [lower_bound, upper_bound]
            variable_definition = self.variables[name] 
        
        elif name not in self.variables[type]:
            variable_definition = self.model.problem.Variable(name + "_" + type, lb = lower_bound, ub = upper_bound, type = vartype)
            self.model.add_cons_vars(variable_definition)
            self.variables[name] = [lower_bound, upper_bound]
            
        return variable_definition
        
        
    def build_constraint(self, constraint_expression, type, lower_bound, upper_bound, object, coef = {}):
        '''Create constraints for the COBRA model
        'type' (Python object, string): The type of the constraint that will be created 
        'lower_bound' (Python object, float): The lower bound value for the added constraint
        'upper_bound' (Python object, float): The upper bound value for the added constraint
        'coef' (Python object, dictionary): The set of coefficients that define the COBRA model
        'object' (Python object, string): The variable name when the name is defined
        '''
        import sympy 
        
        
        # assign a constraint name based upon the passed arguments
        if self.constraint_types[type] == "none":
            count = len(self.constraints[type])
            name = str(count + 1)
        elif self.constraint_types[type] == "string":
            name = object
        else:
            self.constraint_types[type] = coef
            name = object.id
                   
        # add an optlang constraint, when the constraint is undefined 
        if name in self.constraints[type]:
            print('ERROR: The {} constraint value of {} is being overwritten.'.format(name, constraints[type][name]))
        elif name not in self.constraints[type]:
            constraint_name = '{}_{}'.format(name, type)
            constraint_definition = self.model.problem.Constraint(constraint_expression, lb = lower_bound, ub = upper_bound, name = constraint_name)
            self.model.add_cons_vars(constraint_definition)
            if len(coef) > 0:
                constraint_definition.set_linear_coefficients(coef)
                
            self.model.solver.update()
            self.constraints[constraint_name] = [lower_bound, upper_bound]
        
        return constraint_definition
    
    
    def all_variables(self):
        '''Quantify the variables in the class
        '''
        vars = {}
        for child in self.childpkgs:
            for type in child.variables:
                vars[type] = child.variables[type]

        for type in self.variables:
            vars[type] = self.variables[type]
        
        return vars
    
    
    def all_constraints(self):
        '''Quantify the constraints in the class
        '''
        consts = {}
        for child in self.childpkgs:
            for type in child.constraints:
                consts[type] = child.constraints[type]

        for type in self.constraints:
            consts[type] = self.constraints[type]
            
        return consts
    
    
    def clear(self):
        '''Remove all variables from the model instance
        '''
        objects_to_remove = []
        
        for type, value in self.variables.items():
            for object in value:
                objects_to_remove.append(self.variables[type][object])
                
        for type, value in self.constraints.items():
            for object in value:
                objects_to_remove.append(self.constraints[type][object])
                
        # remove variables and constants from the COBRA model 
        self.model.remove_cons_vars(objects_to_remove)
        self.variables = {}
        self.constraints = {}
    
    
# ---------------------------------------------- Revbin -------------------------------------------------

# The base class for FBA packages is inherited
class RevBinPkg(BaseFBAPkg):
    '''Note -> revbin = 0 describes reversibility, while revbin = 1 describes forward progression
    '''
    def __init__(self, model):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''
        BaseFBAPkg.__init__(self, model = model, name = "reversible binary", variable_types = {"revbin":"reaction"}, constraint_types = {"revbinF":"reaction", "revbinR":"reaction"})
    

    def build_constraint(self, object):
        '''Build constraints through the inherited function and the calculated coefficient fluxes
        'object' (Python object, string): The variable name when the name is defined
        '''
        if object.id not in self.variables['revbin']:
            self.variables['revbin'][object.id] = 'test'
        
        constraint_expression = -1000 * revbin(i) + forv(i) <= 0
        coef = {self.variables["revbin"][object.id]:-1000, object.forward_variable:1}
        built_forward_constraint = BaseFBAPkg.build_constraint(self, constraint_expression = constraint_expression, type = "revbinF", lower_bound = None, upper_bound = 1000, coef = {}, object = object)
        
        constraint_expression = 1000 * revbin(i) + revv(i) <= 1000
        coef = {self.variables["revbin"][object.id]:1000, object.reverse_variable:0}
        built_backward_constraint = BaseFBAPkg.build_constraint(self, constraint_expression = constraint_expression, type = "revbinR", lower_bound = None, upper_bound = 1000, coef = {}, object = object)
        
        return built_backward_constraint
    
    
    def build_package(self, filter = model.reactions):
        '''Build variables and constraints through the inherited function
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        '''
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if reaction.id not in filter:
                BaseFBAPkg.build_variable(self, type = "revbin", lower_bound = 0, upper_bound = 1, vartype = "binary", object = reaction)
                self.build_constraint(reaction)
                
                
# ------------------------------------------ Simple Thermo package ------------------------------------------                

# The base class for FBA packages is inherited
class SimpleThermoPkg(BaseFBAPkg):
    def __init__(self, model):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''
        BaseFBAPkg.__init__(self, model = model, name = "simple thermo", variable_types = {"potential":"metabolite"}, constraint_types = {"thermo":"reaction"})
        self.childpkgs["reversible binary"] = RevBinPkg(model)
    

    def build_constraint(self, object):# Gibbs: dg = Sum(st(i,j)*p(j))
        '''Build constraints through the inherited function and the calculated variable coeffiients 
        'object' (Python object, string): The variable name when the name is defined
        '''
        
        # 0 <= 1000*revbin(i) + Sum(st(i,j)*p(j)) <= 1000
        '''coef = {self.childpkgs["reversible binary"].variables["revbin"][object.id] : 1000}
        for metabolite in object.metabolites:
            coef[self.variables["potential"][metabolite.id]] = object.metabolites[metabolite]'''
            
        return BaseFBAPkg.build_constraint(self, type = "thermo", lower_bound = 0, upper_bound = 1000, coef = {}, object = object)

        
    def build_package(self, filter = model.reactions):
        '''Build variables and constraints through the inherited function
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        '''
        self.childpkgs["reversible binary"].build_package(filter)
        for metabolite in self.model.metabolites:
            BaseFBAPkg.build_variable(self, type = "potential", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = metabolite)
            
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if reaction.id not in filter:
                self.build_constraint(reaction)
                
                
# ------------------------------------------ Full Thermo package ------------------------------------------



# The base class for FBA packages is inherited
class FullThermoPkg(BaseFBAPkg):
    def __init__(self, model):
        '''Redefining the inherited __init__ function and importing thermodynamic data
        'model' (COBRA object): The COBRApy FBA model
        '''     
        from numpy import log as ln
        #import modelseedpy
        import pandas
        
        #modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
        #modelseed = modelseedpy.biochem.from_local(modelseed_path)
        
        # import the Gibbs data from the TMFA supplementary excel file
        reactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')
        reactions_data.columns = reactions_data.iloc[0]
        reactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)
        reactions_data.head()
        reactions_dict = {}
        for index, row in reactions_data.iterrows():
            reaction_abbreviation = reactions_data.at[index, 'iJR904 Abbreviation']
            reaction_name = reactions_data.at[index, 'Reaction name']
            reaction_gibbs = reactions_data.at[index, 'Estimated Gibbs free energy change of reaction (kcal/mol)']
            
            reactions_dict[reaction_abbreviation] = {'name': reaction_name, 'gibbs': reaction_gibbs}
        
        self.thermo_reactions = reactions_dict
        
        compounds_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')
        compounds_dict = {}
        for index, row in compounds_data.iterrows():
            compound_abbreviation = compounds_data.at[index, 'iJR904 Abbreviation']
            compound_name = compounds_data.at[index, 'Compound Name']
            compound_gibbs = compounds_data.at[index, 'Estimated Gibbs free energy of formation (kcal/mol)']
            
            compounds_dict[compound_abbreviation] = {'name': compound_name, 'gibbs': compound_gibbs}
        
        self.thermo_compounds = compounds_dict           
            
        # execute the base __init__ file
        BaseFBAPkg.__init__(self, model = model, name = "full thermo", variable_types = {"lnconc":"metabolite", 'thermo': 'reactions'}, constraint_types = {"potentialc":"metabolite", 'thermo': 'metabolite'})
        self.childpkgs["simple thermo"] = SimpleThermoPkg(model)
    
        self.validate_parameters(params = self.parameters, required = [], defaults = {
            "default_max_conc": 20,
            "default_min_conc": 0.001,
            "custom_concentration_constraints": [{"cpd000027_e0": ["10","10"],
                                                'cpd00011_e0': ['20', '24'], # as bicarbonate, E. B. Brown and Richard L. Clancy, 1964
                                                'cpd00067': ['0.0053', '0.0053'], # Battaglia, Hellegers, & Seeds, 1965
                                                'cpd00007': ['0.672', '0.672'] # 0.3 mL / dL serum O2 concentration
                                                 }],                                   
            "compartment_potential": {"c0": 25, 'e0': 20}
        })
    
    
    def build_variable(self, object): #, modelseed = modelseed):
        '''Build variables through the inherited function
        'object' (Python object, string): The variable name when the name is defined
        '''       
        if object.id in self.parameters["custom_concentration_constraints"]:
            lb = ln(self.parameters["custom_concentration_constraints"][object.id][0])
            ub = ln(self.parameters["custom_concentration_constraints"][object.id][1])
        else:
            lb = ln(self.parameters["default_min_conc"])
            ub = ln(self.parameters["default_max_conc"])
            
        gibbs = self.thermo_reactions[object.id]
        print(gibbs)
            
        return BaseFBAPkg.build_variable(self, type = "lnconc", lower_bound = lb, upper_bound = ub, vartype = "continuous", object = object)
    
    
    def build_constraint(self, object, type = 'thermo', odelseed = modelseed):
        '''Build constraints through the inherited function and the calculated variable coeffiients 
        'object' (Python object, string): The variable name when the name is defined
        '''
        # Equation 14 in the TMFA paper, with the addition of the (charge * compartment_potential) term?
        # potential(i) (KJ/mol) = deltaG^naut(i) (KJ/mol) + R * T(K) * Sum(x_i * lnconc(i * gamma)) + charge(i) * compartment_potential
        constant = 20
        #coef = {self.childpkgs["potential"].variables["revbin"][object.id] : 1000}
        
        constraint_name = '{}_{}'.format(object.id, type)
        if constraint_name not in model.constraints:
            built_constraint = BaseFBAPkg.build_constraint(self, type = type, lower_bound = constant, upper_bound = constant, coef = {}, object = object)
            
        else:
            print('ERROR: The {} contraint already exists in the model: {}'.format(constraint_name, model.constraints[constraint_name]))
            built_constraint = None
        
        return built_constraint

        
    def build_package(self, filter = model.reactions):
        '''Create the final model package
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        Notes - The concentrations are expressed in millimolar
        '''        
        # Extend the custom concentration for H+, CO2, O2
        self.childpkgs["simple thermo"].build_package(filter)
        
        # The concentration variable and potential constraint are built
        for metabolite in self.model.metabolites:
            BaseFBAPkg.build_variable(self, type = "thermo", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = metabolite)
            
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if reaction.id not in filter:
                self.build_constraint(reaction)


#------------------------------------------------ Test ---------------------------------------------------

# Base FBA Package
announcement = 'BaseFBApkg'
print(announcement, '\n', '='*len(announcement))


# __init__ 
print('\n+ __init__\n')
base = BaseFBAPkg(model, 'test_model')      
print('model used: ', base.model)
print('instance name: ', base.name)
print('variable types: ', base.variable_types)
print('constraint types: ', base.constraint_types)


# validate_parameters 
print('\n+ validate_parameters')
params = {'a':2, 'b':4, 'c':3}
required = ['a', 'b', 'c']
defaults = {'a':1, 'b':1, 'c':1, 'd':1}

print('parameters before: ', base.parameters)
base.validate_parameters(params, required, defaults)
print('parameters after: ', base.parameters)


# build_variable
print('\n+ build_variable')
print('variables before: ', base.variables)
base.build_variable(type = 'concentration', lower_bound = 10, upper_bound = 133, vartype = 'continuous', object = model.reactions.get_by_id('PFK'))
print('variables after: ', base.variables)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', base.constraints)
base.build_constraint(type = 'concentration', lower_bound = 10, upper_bound = 133, object = model.reactions.get_by_id('PFK'))
print('constraints after: ', base.constraints)


# all_variables
print('\n+ all_variables')
print('model variables: ', base.all_variables)


# all_constraints
print('\n+ all_constraints')
print('model constraints: ', base.all_constraints)


# clear
print('\n+ clear')
base.clear
print('model variables: ', base.variables)
print('model constraints: ', base.constraints)


# Revbin
announcement = 'Revbin'
print('\n', announcement, '\n', '='*len(announcement))


# __init__
print('\n+ __init__')
revbin = RevBinPkg(model)
print('model used: ', revbin.model)
print('instance name: ', revbin.name)
print('variable types: ', revbin.variable_types)
print('constraint types: ', revbin.constraint_types)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', revbin.constraints)
revbin.build_constraint(object = model.reactions.get_by_id('PFK'))
print('constraints after: ', revbin.constraints)


# build_package
print('\n+ build_package')
print('constraints before: ', revbin.constraints)
revbin.build_package()
print('constraints after: ', revbin.constraints)


# Simple Thermo
announcement = 'Simple Thermo'
print('\n', announcement, '\n', '='*len(announcement))


# __init__
print('\n+ __init__')
simple_thermo = SimpleThermoPkg(model)
print('model used: ', simple_thermo.model)
print('instance name: ', simple_thermo.name)
print('variable types: ', simple_thermo.variable_types)
print('constraint types: ', simple_thermo.constraint_types)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', simple_thermo.constraints)
simple_thermo.build_constraint(object = model.reactions.get_by_id('PFK'))
print('constraints after: ', simple_thermo.constraints)


# build_package
print('\n+ build_package')
print('constraints before: ', simple_thermo.constraints)
simple_thermo.build_package()
print('constraints after: ', simple_thermo.constraints)


# Full Thermo
announcement = 'Full Thermo'
print('\n', announcement, '\n', '='*len(announcement))


# __init__
print('\n+ __init__')
full_thermo = FullThermoPkg(model)
print('model used: ', full_thermo.model)
print('instance name: ', full_thermo.name)
print('variable types: ', full_thermo.variable_types)
print('constraint types: ', full_thermo.constraint_types)


# build_variable
print('\n+ build_variable')
print('variables before: ', full_thermo.variables)
full_thermo.build_variable(object = model.reactions.get_by_id('PFK'))
print('variables after: ', full_thermo.variables)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', full_thermo.constraints)
full_thermo.build_constraint(object = model.reactions.get_by_id('PFK'))
print('constraints after: ', full_thermo.constraints)


# build_package
print('\n+ build_package')
print('constraints before: ', full_thermo.constraints)
full_thermo.build_package()
print('constraints after: ', full_thermo.constraints)

BaseFBApkg 

+ __init__

model used:  e_coli_core
instance name:  test_model
variable types:  {'concentration': 'float', 'lnconc': 'float'}
constraint types:  {'concentration': 'float'}

+ validate_parameters
parameters before:  {}
All required parameters are present.
parameters after:  {'a': 2, 'b': 4, 'c': 3, 'd': 1}

+ build_variable
variables before:  {'concentration': {}, 'lnconc': {}}
variables after:  {'concentration': {}, 'lnconc': {}, 'PFK': [10, 133]}

+ build_constraint
constraints before:  {'concentration': {}}
constraints after:  {'concentration': {}, 'PFK_concentration': [10, 133]}

+ all_variables
model variables:  <bound method BaseFBAPkg.all_variables of <__main__.BaseFBAPkg object at 0x000001CA13B60808>>

+ all_constraints
model constraints:  <bound method BaseFBAPkg.all_constraints of <__main__.BaseFBAPkg object at 0x000001CA13B60808>>

+ clear
model variables:  {'concentration': {}, 'lnconc': {}, 'PFK': [10, 133]}
model constraints:  {'concentration': {}, 'PFK_conce

# Brainstorming

In [2]:
dictionary = {'a':1, 'b':2}
require = {'a':2, 'c':2}
set_1 = set(require) - set(dictionary)
print(set_1)

{'c'}


In [5]:
defaults = {"custom_concentration_constraints": {"cpd000027_e0": ["10","10"],
                                                'cpd000033_c0': ['3', '4']}}
print(defaults["custom_concentration_constraints"]['cpd000033_c0'][0])

3


In [16]:
import pandas
compound_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')
gibbs_compounds = compound_data['Estimated Gibbs free energy of formation (kcal/mol)']
# = thermodynamic_data['Compounds']
display(gibbs_compound_data)
#display(thermodynamic_data)

0     -152.39300
1     -181.65000
2      -79.54030
3     -587.09200
4       -7.09536
         ...    
623   -181.77700
624   -182.11500
625      Unknown
626          NaN
627          NaN
Name: Estimated Gibbs free energy of formation (kcal/mol), Length: 628, dtype: object

In [78]:
import cobra
import modelseedpy

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

reaction = 'PFK'
reaction_name = model.reactions.get_by_id(reaction).name
modelseed_reaction = modelseed.get_seed_reaction(reaction_name)

print(modelseed_reaction)

None


In [75]:
import modelseedpy

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)

dir(modelseed)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'compound_aliases',
 'compound_structures',
 'compounds',
 'get_attribute',
 'get_formula',
 'get_mapping_external_to_seed',
 'get_name',
 'get_non_obsolete',
 'get_seed_compound',
 'get_seed_compound_by_alias',
 'get_seed_reaction',
 'get_seed_reaction_by_alias',
 'reaction_aliases',
 'reaction_ecs',
 'reactions',
 'summary']

In [92]:
import pandas

reactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')
reactions_data.columns = reactions_data.iloc[0]
reactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)
reactions_data.head()

,iJR904 Abbreviation,Reaction name,Estimated Gibbs free energy change of reaction (kcal/mol),Standard uncertainty in estimate (kcal/mol)
1,ALAR,alanine racemase,0,0.50000
2,ASNN,L-asparaginase,-4.20409,1.16973
3,ASNS2,asparagine synthetase,-2.54684,1.40088
4,ASNS1,asparagine synthase (glutamine-hydrolysing),-6.75092,1.04603
5,ASPT,L-aspartase,2.40657,1.02896
